## Installations

In [1]:
!pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [4]:
!pip install -U google-ai-generativelanguage==0.6.18 langchain-google-genai

## Imports

In [37]:
import os
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
import getpass
from langgraph.checkpoint.memory import MemorySaver

## Setting API Keys

In [30]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

os.environ["TAVILY_API_KEY"] = getpass.getpass()

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

## Defining The Tool

In [31]:
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Kolkata?")
print(search_results)

tools = [search]

{'query': 'What is the weather in Kolkata?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Kolkata', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kolkata', 'region': 'West Bengal', 'country': 'India', 'lat': 22.5697, 'lon': 88.3697, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1754229724, 'localtime': '2025-08-03 19:32'}, 'current': {'last_updated_epoch': 1754229600, 'last_updated': '2025-08-03 19:30', 'temp_c': 29.0, 'temp_f': 84.2, 'is_day': 0, 'condition': {'text': 'Patchy light rain with thunder', 'icon': '//cdn.weatherapi.com/weather/64x64/night/386.png', 'code': 1273}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 191, 'wind_dir': 'SSW', 'pressure_mb': 1005.0, 'pressure_in': 29.68, 'precip_mm': 0.11, 'precip_in': 0.0, 'humidity': 94, 'cloud': 75, 'feelslike_c': 34.2, 'feelslike_f': 93.5, 'windchill_c': 29.2, 'windchill_f': 84.5, 'heatindex_c': 34.6, 'heatindex_f': 94.3, 'dewpoint_c': 24.9, 'dewpoint_f'

## Using The LLM Gemini 2.5 Flash

In [27]:
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

## Calling The LLM

In [6]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

## Enabling The LLM To Do Tool Calling

In [33]:
model_with_tool = model.bind_tools(tools)

## Calling the LLM again
###This time, tool calling is enabled with the LLM but no tool call is there as the query asked does not require any web-search.

In [34]:
query = "Hi!"
response = model_with_tool.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


## Calling The LLM With An Input That Expects Tool Call
### There is now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

In [35]:
query = "Search for the weather in Kolkata."
response = model_with_tool.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool call: {response.tool_calls}")

Message content: 

Tool call: [{'name': 'tavily_search', 'args': {'query': 'weather in Kolkata'}, 'id': 'b53b1394-c248-47ca-b8e7-34cacd3fb609', 'type': 'tool_call'}]


### In order to actually call the tool, we are creating our agent.

## Create The Agent

In [26]:
agent_executor = create_react_agent(model, tools)

## Running The Agent Where Tool Call Is Not Required

In [11]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


## Running The Agent Where Tool Call Is Required

In [13]:
input_message = {"role": "user", "content": "Search for the weather in Kolkata."}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Kolkata.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (29e3d959-0e5d-423d-844d-561d2fcf0bec)
 Call ID: 29e3d959-0e5d-423d-844d-561d2fcf0bec
  Args:
    query: weather in Kolkata
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Kolkata", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Kolkata", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Kolkata', 'region': 'West Bengal', 'country': 'India', 'lat': 22.5697, 'lon': 88.3697, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1754230546, 'localtime': '2025-08-03 19:45'}, 'current': {'last_updated_epoch': 1754230500, 'last_updated': '2025-08-03 19:45', 'temp_c': 29.3, 'temp_f': 84.7, 'is_day': 0, 'condition': {'text': 'Patchy light 

## Streaming Messages

In [14]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Kolkata.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (f245bf51-9e03-4658-bb4a-877d4e1985fe)
 Call ID: f245bf51-9e03-4658-bb4a-877d4e1985fe
  Args:
    query: weather in Kolkata
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Kolkata", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Kolkata", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Kolkata', 'region': 'West Bengal', 'country': 'India', 'lat': 22.5697, 'lon': 88.3697, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1754230546, 'localtime': '2025-08-03 19:45'}, 'current': {'last_updated_epoch': 1754230500, 'last_updated': '2025-08-03 19:45', 'temp_c': 29.3, 'temp_f': 84.7, 'is_day': 0, 'condition': {'text': 'Patchy light 

## Streaming Tokens

In [15]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The current weather in Kolkata, India,| as of 7:45 PM on August 3, 2025, is 29.3°C (84.7°F) with patchy light rain and thunder. It feels like 34|.9°C (94.9°F) due to 94% humidity. The wind is blowing from the SSW at 16.9 kph (10.5 mph).

The forecast for today in| Kolkata is partly cloudy with a maximum temperature of 33°C and a minimum temperature of 29°C.|

## Adding Memory

In [36]:
memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}  # Keeping thread_id as "abc123"

In [20]:
input_message = {"role": "user", "content": "Hi, I'm Rup!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values" # Keeping the same thread_id as "abc123"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Rup!
================================== Ai Message ==================================

Hello Rup! How can I help you today?


In [21]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values" # Keeping the same thread_id as "abc123", so the agent can remember my name
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Rup.


In [22]:
config = {"configurable": {"thread_id": "xyz123"}} # Keeping a different thread_id from "abc123", so the agent cannot remember my name

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not know your name. I am a large language model, able to communicate in response to a wide range of prompts and questions, but I have no memory of past conversations or personal information.
